<a href="https://colab.research.google.com/github/pgurazada/large-models-on-colab/blob/main/BLOOM_BigScience_bitsandbytes_int8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace meets `bitsandbytes` for lighter models on GPU for inference

Bloom: https://huggingface.co/docs/transformers/model_doc/bloom

| Code Credits | Link |
| ----------- | ---- |
| 🎉 Repository | [![GitHub Repository](https://img.shields.io/github/stars/TimDettmers/bitsandbytes?style=social)](https://github.com/TimDettmers/bitsandbytes) |
| 🔥 Discover More Colab Notebooks | [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-black?style=flat-square&logo=github)](https://github.com/R3gm/InsightSolver-Colab/) |



You can run your own 8-bit model on any HuggingFace 🤗 model with just few lines of code. Install the dependencies below first!


In [1]:
! pip install -q transformers accelerate bitsandbytes xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00


## Hardware requirements 🔨

To run properly this feature you need to have GPU that supports 8-bit operation modules. Currently, Turing and Ampere GPUs (RTX20s, RTX30s, A40-A100, T4+) are supported, which means on colab we need to use a T4 GPU for this feature. You can check that using this code snippet and make sure you are using a supported GPU

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Sep  6 04:49:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Here we are using a `Tesla T4` GPU that should support 8-bit tensor cores! We are good to go 🚀

## Utility variables & functions 🧰

In [3]:
name = "bigscience/bloom-3b"
#name = "facebook/opt-1.3b"
text = "Hello my name is"
max_new_tokens = 20

In [4]:
def generate_from_model(model, tokenizer):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda())
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

## Use 8bit models and `pipeline` 🤗

You can use 8bit quantized models together with `pipeline` as follows:

In [5]:
from transformers import pipeline

In [6]:
pipe = pipeline(
    model=name,
    model_kwargs= {"device_map": "auto", "load_in_8bit": True},
    max_new_tokens=max_new_tokens
)

Let's check the output!

In [7]:
pipe(text)

[{'generated_text': 'Hello my name is John and I am a student at the University of the West of England. I am currently studying for'}]

## Use 8bit models and `.generate` 📖

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [9]:
model_8bit = AutoModelForCausalLM.from_pretrained(
    name, device_map="auto", load_in_8bit=True
)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(name)

In [11]:
generate_from_model(model_8bit, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Hello my name is John and I am a student at the University of the West of England. I'

In [12]:
# memory size
mem_int8 = model_8bit.get_memory_footprint()

In [13]:
print("Memory footprint int8 model: {} ".format(mem_int8))

Memory footprint int8 model: 3645818880 


Let's compare the qualitative results between our quantized model and the original model

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [15]:
name = "bigscience/bloom-3b"
#name = "facebook/opt-1.3b"
text = "Hello my name is"
max_new_tokens = 20

In [16]:
def generate_from_model(model, tokenizer):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda())
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

In [17]:
model_native = AutoModelForCausalLM.from_pretrained(
    name, device_map="auto", torch_dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained(name)

'Hello my name is John and I am a student at the University of the West Indies. I am'

In [18]:
generate_from_model(model_native, tokenizer)

'Hello my name is John and I am a student at the University of the West Indies. I am'

In [19]:
# memory size
mem_fp16 = model_native.get_memory_footprint()

In [20]:
print("Memory footprint fp16 model: {}".format( mem_fp16))

Memory footprint fp16 model: 6005114880


## Memory footprint comparison 🪶

In [22]:
print("Memory footprint int8 model: {} | Memory footprint fp16 model: {} | Relative difference: {}".format(mem_int8, mem_fp16, mem_fp16/mem_int8))

Memory footprint int8 model: 3645818880 | Memory footprint fp16 model: 6005114880 | Relative difference: 1.6471237539918604


We saved 1.65x memory for a 3-billion parameters models! Note that internally we replace all the linear layers by the ones implemented in `bitsandbytes`. By scaling up the model the number of linear layers will increase therefore the impact of saving memory on those layers will be huge for very large models. For example quantizing BLOOM-176 (176 Billion parameter model) gives a gain of 1.96x memory footprint which can save a lot of compute power in practice.